In [3]:
from bqt import bqt
import spotify_confidence as conf
import pandas as pd

## Determine count of users in the past 21 days who visited npv and episode page of chapters enabled episodes

In [4]:
episode_npv_visitors='''
WITH dd1 AS  
    (select distinct uri from `spotify-entities.show.*`,unnest(name) where parse_DATE('%Y%m%d',_table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day) and text not in ('On Purpose with Jay Shetty',
    'Impaulsive with Logan Paul',"Andrew Schulz's Flagrant with Akaash Singh",
    "Huberman Lab",
    "Andrew Schulz’s Flagrant 2 with Akaash Singh (New Chapter)",
    "Andrew Schulz's Flagrant 2 with Akaash S",
    'Bad Friends',
    'All-In with Chamath, Jason, Sacks & Friedberg',
    'The Morning Toast',
    'The Bill Simmons Podcast',
    'Skip and Shannon: Undisputed',
    'Fantasy Footballers - Fantasy Football Podcast',
    "Bussin' With The Boys",
    'The Tim Ferriss Show',
    'The NoSleep Podcast',
    'The Weekly Planet',
    'Lights Camera Barstool',
    'KFC Radio',
    'Mile Higher',
    'Ear Biscuits',
    'Pardon My Take',
    'Token CEO',
    'BFFs featuring Josh Richards and Dave Portnoy',
    'The Vergecast',
    'Macrodosing: Arian Foster and PFT Commenter',
    'My Brother, My Brother And Me',
    'Breaking Points with Krystal and Saagar',
    "MrCreepyPasta's Storytime",
    'Spittin Chiclets',
    'Token CEO',
    'BFFs with Dave Portnoy, Josh Richards, and Brianna Chickenfry',
    'The School of Greatness',
    'We Might Be Drunk',
    'TigerBelly',
    'BFFs featuring Josh Richards and Dave Portnoy',
    'Pardon My Take',
    'The Jordan B. Peterson Premium Podcast (🔓)',
    'The Dave Portnoy Show with Eddie & Co',
    'Unexplained Encounters (🔓)',
    'The Breakfast Club',
    'Unexplained Encounters')),
    episodes_w_chapters as
    (SELECT distinct
            e.uri AS episode_uri
    FROM `spotify-entities.episode_experimental.*` e, unnest(e.name) as name
    JOIN `spotify-entities.show.*` s
    ON s.uri = e.showRelation.show.uri
    WHERE chapter IS NOT NULL
    AND ARRAY_LENGTH(chapter) > 0 and e.isMusicAndTalk is false
    AND parse_DATE('%Y%m%d',e._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    AND parse_DATE('%Y%m%d',s._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    union all
    SELECT distinct
            e.uri AS episode_uri
    FROM `spotify-entities.episode_experimental.*` e, unnest(e.name) as name
    JOIN dd1 s
    ON s.uri = e.showRelation.show.uri
    JOIN `spotify-podz.creator_episode_segments.creator_episode_segments_*` f
    ON e.uri=f.episode_uri
    WHERE  e.isMusicAndTalk is false
    AND parse_DATE('%Y%m%d',e._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    AND parse_DATE('%Y%m%d',f._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    ),
    end_content_podcast AS
    (SELECT
        playback.playback_id as playback_id,
        content.uri as ep_uri
    FROM `stream-pipelines.stream_days_v1.stream_days_v1_*`
    WHERE parse_DATE('%Y%m%d',_table_suffix) >= DATE_SUB(CURRENT_DATE(), INTERVAL 35 day) and parse_DATE('%Y%m%d',_table_suffix) <= DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    AND content.uri in (select distinct episode_uri from episodes_w_chapters )),
    npv_impression AS 
    (SELECT 
        user_id, 
        playback_id,path,parse_DATE('%Y%m%d',_table_suffix) dat
    FROM `user-behaviour-instrumentation.impression_fact.impression_fact_*`
    WHERE specification_id in ( 'mobile-now-playing-view') and playback_id is not null and user_id is not null and playback_id in  --and id like 'podcast%'
    (select distinct playback_id from end_content_podcast) and specification_id is not null 
    AND parse_DATE('%Y%m%d',_table_suffix) >= DATE_SUB(CURRENT_DATE(), INTERVAL 35 day) and parse_DATE('%Y%m%d',_table_suffix) <= DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)),
   
    viewed_npv_table AS 
    (SELECT 
        user_id, 
        min(dat) exposure_date
    FROM npv_impression,unnest(path)
    WHERE  id like 'podcast%' group by 1
    ),
    
    viewed_episode_page_table AS 
    (SELECT 
            pt.user_id,min(parse_DATE('%Y%m%d',_table_suffix)) exposure_date
            
    FROM `user-behaviour-instrumentation.pagetransition_fact.pagetransition_fact_*` pt
    INNER JOIN episodes_w_chapters ep
    ON pt.entity_uri_safe_decrypted = ep.episode_uri 
    WHERE  parse_DATE('%Y%m%d',_table_suffix) >= DATE_SUB(CURRENT_DATE(), INTERVAL 35 day) and parse_DATE('%Y%m%d',_table_suffix) <= DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
   
    AND pt.page_id = 'podcast/episode' group by 1)
            
    SELECT
        user_id,min(exposure_date) exposure_date
    FROM (select * from viewed_npv_table union all select * from viewed_episode_page_table) group by 1
    '''
episode_npv_visitos_df = bqt.query(episode_npv_visitors)

Running job core-x-insights:8e5da49f-2058-4496-9a91-d2cce4bffcd0
Query done! Processed: 281851.1 G Billed: 281851.1 G Cost: $1,281.71                                                                       
Finished downloading results            


In [8]:
episode_npv_visitos_df.count()

user_id          2742786
exposure_date    2742786
dtype: int64

## Determine unique podcast consumed in the past 21 days by each user

In [7]:
unique_podcast_consumed='''
WITH dd1 AS  
    (select distinct uri from `spotify-entities.show.*`,unnest(name) where parse_DATE('%Y%m%d',_table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day) and text not in ('On Purpose with Jay Shetty',
    'Impaulsive with Logan Paul',"Andrew Schulz's Flagrant with Akaash Singh",
    "Huberman Lab",
    "Andrew Schulz’s Flagrant 2 with Akaash Singh (New Chapter)",
    "Andrew Schulz's Flagrant 2 with Akaash S",
    'Bad Friends',
    'All-In with Chamath, Jason, Sacks & Friedberg',
    'The Morning Toast',
    'The Bill Simmons Podcast',
    'Skip and Shannon: Undisputed',
    'Fantasy Footballers - Fantasy Football Podcast',
    "Bussin' With The Boys",
    'The Tim Ferriss Show',
    'The NoSleep Podcast',
    'The Weekly Planet',
    'Lights Camera Barstool',
    'KFC Radio',
    'Mile Higher',
    'Ear Biscuits',
    'Pardon My Take',
    'Token CEO',
    'BFFs featuring Josh Richards and Dave Portnoy',
    'The Vergecast',
    'Macrodosing: Arian Foster and PFT Commenter',
    'My Brother, My Brother And Me',
    'Breaking Points with Krystal and Saagar',
    "MrCreepyPasta's Storytime",
    'Spittin Chiclets',
    'Token CEO',
    'BFFs with Dave Portnoy, Josh Richards, and Brianna Chickenfry',
    'The School of Greatness',
    'We Might Be Drunk',
    'TigerBelly',
    'BFFs featuring Josh Richards and Dave Portnoy',
    'Pardon My Take',
    'The Jordan B. Peterson Premium Podcast (🔓)',
    'The Dave Portnoy Show with Eddie & Co',
    'Unexplained Encounters (🔓)',
    'The Breakfast Club',
    'Unexplained Encounters')),
    episodes_w_chapters as
    (SELECT distinct
            e.uri AS episode_uri
    FROM `spotify-entities.episode_experimental.*` e, unnest(e.name) as name
    JOIN `spotify-entities.show.*` s
    ON s.uri = e.showRelation.show.uri
    WHERE chapter IS NOT NULL
    AND ARRAY_LENGTH(chapter) > 0 and e.isMusicAndTalk is false
    AND parse_DATE('%Y%m%d',e._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    AND parse_DATE('%Y%m%d',s._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    union all
    SELECT distinct
            e.uri AS episode_uri
    FROM `spotify-entities.episode_experimental.*` e, unnest(e.name) as name
    JOIN dd1 s
    ON s.uri = e.showRelation.show.uri
    JOIN `spotify-podz.creator_episode_segments.creator_episode_segments_*` f
    ON e.uri=f.episode_uri
    WHERE  e.isMusicAndTalk is false
    AND parse_DATE('%Y%m%d',e._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    AND parse_DATE('%Y%m%d',f._table_suffix) = DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    ),
    end_content_podcast AS
    (SELECT
        playback.playback_id as playback_id,
        content.uri as ep_uri
    FROM `stream-pipelines.stream_days_v1.stream_days_v1_*`
    WHERE parse_DATE('%Y%m%d',_table_suffix) >= DATE_SUB(CURRENT_DATE(), INTERVAL 35 day) and parse_DATE('%Y%m%d',_table_suffix) <= DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
    AND content.uri in (select distinct episode_uri from episodes_w_chapters )),
    npv_impression AS 
    (SELECT 
        user_id, 
        playback_id,path,parse_DATE('%Y%m%d',_table_suffix) dat
    FROM `user-behaviour-instrumentation.impression_fact.impression_fact_*`
    WHERE specification_id in ( 'mobile-now-playing-view') and playback_id is not null and user_id is not null and playback_id in  --and id like 'podcast%'
    (select distinct playback_id from end_content_podcast) and specification_id is not null 
    AND parse_DATE('%Y%m%d',_table_suffix) >= DATE_SUB(CURRENT_DATE(), INTERVAL 35 day) and parse_DATE('%Y%m%d',_table_suffix) <= DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)),
   
    viewed_npv_table AS 
    (SELECT 
        user_id, 
        min(dat) exposure_date
    FROM npv_impression,unnest(path)
    WHERE  id like 'podcast%' group by 1
    ),
    
    viewed_episode_page_table AS 
    (SELECT 
            pt.user_id,min(parse_DATE('%Y%m%d',_table_suffix)) exposure_date
            
    FROM `user-behaviour-instrumentation.pagetransition_fact.pagetransition_fact_*` pt
    INNER JOIN episodes_w_chapters ep
    ON pt.entity_uri_safe_decrypted = ep.episode_uri 
    WHERE  parse_DATE('%Y%m%d',_table_suffix) >= DATE_SUB(CURRENT_DATE(), INTERVAL 35 day) and parse_DATE('%Y%m%d',_table_suffix) <= DATE_SUB(CURRENT_DATE(), INTERVAL 14 day)
   
    AND pt.page_id = 'podcast/episode' group by 1),
     cc as (       
    SELECT
        user_id,min(exposure_date) exposure_date
    FROM (select * from viewed_npv_table union all select * from viewed_episode_page_table) group by 1)
        
        SELECT
          a.user_id,count(distinct episode_uri) cnt
        FROM
          `podcast-listening-affinity.nmc_streams.nmc_streams_*` a inner join cc on a.user_id=cc.user_id
        WHERE ms_played > 30000 
        AND parse_DATE('%Y%m%d',_table_suffix) >= exposure_date  and parse_DATE('%Y%m%d',_table_suffix) <= exposure_date + 14 group by 1 
    
    '''
unique_podcast_consumed_df = bqt.query(unique_podcast_consumed)

Running job core-x-insights:c976207a-bdbf-435f-ae71-8ef93feefb3c
Query done! Processed: 287287.1 G Billed: 287287.1 G Cost: $1,306.43                                                              
Finished downloading results            


In [9]:
unique_podcast_consumed_df.count()

user_id    2609731
cnt        2609731
dtype: int64

In [10]:
def fill_with_zeros(df, total_rows, column):
    zeros = zeros = [{column: 0} for i in range(df.count()[column], total_rows)]
    return pd.concat([df, pd.DataFrame.from_dict(zeros)])

fill the users who did not consume podcast with 0 counts and calculate sample size with 1% mde,1% alpha and 80% power

In [12]:
unique_podcast_consumed_df1 = fill_with_zeros(unique_podcast_consumed_df, 2742786, 'cnt')
relative_mde = 0.01 
absolute_mde = unique_podcast_consumed_df1.cnt.mean() * relative_mde
alpha = 0.01
power = 0.8
conf.SampleSize.continuous(average_absolute_mde=absolute_mde,
                           baseline_variance=unique_podcast_consumed_df1.cnt.var(),
                           alpha=alpha,
                           power=power,
                           treatments=2,
                           comparisons='control_vs_all',
                           treatment_allocations=[0.5, 0.5],
                           bonferroni_correction=True)

(784011, array([392006, 392006]), array([0.5, 0.5]))

so for above metric we have more than required users avialable for the experiment

## Determine WAU at d14 after exposure

In [13]:
# I have saved the 21 day npv and episode page visitors in the table `core-x-insights.rahul.npv_episode_visitors`
wau='''    
       SELECT 
    COUNT(DISTINCT cc.user_id) AS distinct_users
    
FROM `core-x-insights.rahul.npv_episode_visitors` cc left join `experience-mission-metrics.activity.activity` a
 ON TO_HEX(a.user_id) = cc.user_id
WHERE DATE(_PARTITIONTIME) = DATE_ADD(EXPOSURE_DATE, INTERVAL 14 day)
and wau is true
    
    '''
wau_df = bqt.fast_query(wau)

Attempting dry-run before executing full query...

SELECT COUNT(DISTINCT cc.user_id) AS distinct_users
FROM `core-x-insights.rahul.npv_episode_visitors` cc
LEFT JOIN `experience-mission-metrics.activity.activity` a ON TO_HEX(a.user_id) = cc.user_id
WHERE DATE(_PARTITIONTIME) = DATE_ADD(EXPOSURE_DATE, INTERVAL 14 DAY)
  AND wau IS TRUE
DRY RUN: Would Process: 27372.6 G Would Cost: $124.48
Staging bucket: gs://fastbqt-staging-eu-core-x-insights
Job core-x-insights:a73d3e90-4230-4c61-9ad9-b3dffa2efe18 submitted ...
Running job core-x-insights:a73d3e90-4230-4c61-9ad9-b3dffa2efe18
Job finished successfully!                                   
Got 1 rows (0.0)
Compressing and preparing files ...
1 chunks (43.0) will be downloaded!
Time elapsed: 0:45:25.479210


In [16]:
wau_df

,distinct_users
0,2572605


In [18]:
relative_mde = 0.0011
absolute_percentage_mde = (2572605 / 2742786) * relative_mde
conf.SampleSize.binomial(absolute_percentage_mde=absolute_percentage_mde,
                         baseline_proportion=2572605 / 2742786,
                         alpha=alpha,
                         power=power,
                         treatments=2,
                         comparisons='control_vs_all',
                         treatment_allocations=[0.5, 0.5],
                         bonferroni_correction=True)

(2553978, array([1276989, 1276989]), array([0.5, 0.5]))

so total users avaiable for experiment is 2742786 and its below required sample size, so we are good with this metric

## Determine podcast consumtion at d14 after exposure

In [14]:
podcast_consumption='''
       SELECT 
   AVG(coalesce(podcast_min_played_last14d,0)) AS AVG,
    STDDEV(coalesce(podcast_min_played_last14d,0)) AS std
    
FROM `core-x-insights.rahul.npv_episode_visitors` cc left join `experience-mission-metrics.activity.activity` a
 ON TO_HEX(a.user_id) = cc.user_id
WHERE DATE(_PARTITIONTIME) = DATE_ADD(EXPOSURE_DATE, INTERVAL 14 day)

    
    '''
podcast_consumption_df = bqt.fast_query(podcast_consumption)

Attempting dry-run before executing full query...

SELECT AVG(coalesce(podcast_min_played_last14d, 0)) AS AVG,
       STDDEV(coalesce(podcast_min_played_last14d, 0)) AS std
FROM `core-x-insights.rahul.npv_episode_visitors` cc
LEFT JOIN `experience-mission-metrics.activity.activity` a ON TO_HEX(a.user_id) = cc.user_id
WHERE DATE(_PARTITIONTIME) = DATE_ADD(EXPOSURE_DATE, INTERVAL 14 DAY)
DRY RUN: Would Process: 37457.2 G Would Cost: $170.34
Staging bucket: gs://fastbqt-staging-eu-core-x-insights
Job core-x-insights:4eb2a0d6-3d1f-41b2-98ab-ba9cb105afa5 submitted ...
Running job core-x-insights:4eb2a0d6-3d1f-41b2-98ab-ba9cb105afa5
Job finished successfully!                                   
Got 1 rows (0.0)
Compressing and preparing files ...
1 chunks (62.0) will be downloaded!
Time elapsed: 0:45:01.704711


In [20]:
podcast_consumption_df.std

<bound method NDFrame._add_numeric_operations.<locals>.std of           AVG          std
0  724.199872  1006.215963>

In [38]:

relative_mde = 0.0058
absolute_mde = 724.2 * relative_mde
alpha = 0.01
power = 0.8
conf.SampleSize.continuous(average_absolute_mde=absolute_mde,
                           baseline_variance=1006.22*1006.22,
                           alpha=alpha,
                           power=power,
                           treatments=2,
                           comparisons='control_vs_all',
                           treatment_allocations=[0.5, 0.5],
                           bonferroni_correction=True)

(2680880, array([1340440, 1340440]), array([0.5, 0.5]))

had to increase the NIM threshold to 0.58% in order to meet the required sample size criteria 

## Determine overall consumtion at d14 after exposure

In [15]:
overall_consumption='''
       SELECT 
   AVG(coalesce(min_played_last14d,0)) AS AVG,
    STDDEV(coalesce(min_played_last14d,0)) AS std
    
FROM `core-x-insights.rahul.npv_episode_visitors` cc left join `experience-mission-metrics.activity.activity` a
 ON TO_HEX(a.user_id) = cc.user_id
WHERE DATE(_PARTITIONTIME) = DATE_ADD(EXPOSURE_DATE, INTERVAL 14 day)

    
    '''
overall_consumption_df = bqt.fast_query(overall_consumption)

Attempting dry-run before executing full query...

SELECT AVG(coalesce(min_played_last14d, 0)) AS AVG,
       STDDEV(coalesce(min_played_last14d, 0)) AS std
FROM `core-x-insights.rahul.npv_episode_visitors` cc
LEFT JOIN `experience-mission-metrics.activity.activity` a ON TO_HEX(a.user_id) = cc.user_id
WHERE DATE(_PARTITIONTIME) = DATE_ADD(EXPOSURE_DATE, INTERVAL 14 DAY)
DRY RUN: Would Process: 37457.2 G Would Cost: $170.34
Staging bucket: gs://fastbqt-staging-eu-core-x-insights
Job core-x-insights:f99de3cb-fe67-404b-89c5-a69e04e62483 submitted ...
Running job core-x-insights:f99de3cb-fe67-404b-89c5-a69e04e62483
Job finished successfully!                                   
Got 1 rows (0.0)
Compressing and preparing files ...
1 chunks (66.0) will be downloaded!
Time elapsed: 0:53:07.019614


In [26]:
overall_consumption_df


,AVG,std
0,1665.063757,1481.992214


In [28]:
relative_mde = 0.005
absolute_mde = 1665.064 * relative_mde
alpha = 0.01
power = 0.8
conf.SampleSize.continuous(average_absolute_mde=absolute_mde,
                           baseline_variance=1482*1482,
                           alpha=alpha,
                           power=power,
                           treatments=2,
                           comparisons='control_vs_all',
                           treatment_allocations=[0.5, 0.5],
                           bonferroni_correction=True)

(1480332, array([740166, 740166]), array([0.5, 0.5]))

we are well within sample size requirement for this metric

Lets try including all the required metric within conf calculation to see overall sample size requirement

In [35]:
df = pd.DataFrame(
    columns=["metric_name", "binary", "avg", "var", "mde", "nim", "preference"],
    data=[
        ["unique_podcast_consume", False, 17.51, 514.8, 0.01, None, "increase"],
        ["wau_14d", True, 0.938, 0.058, None,0.0012, "increase"],
        ["podcast_consumed_14d", False, 724, 1006*1006, None,0.0062, "increase"],
        ["overall_consumed_14d", False, 1665, 1482*1482,None, 0.005, "increase"],
    ],
)

ssc = conf.SampleSizeCalculator(
    data_frame=df,
    point_estimate_column="avg",
    var_column="var",
    metric_column="metric_name",
    is_binary_column="binary",
    interval_size=0.99,
    power=0.8,
    correction_method='bonferroni',
)
treatment_weights = [5000, 5000]
ss = ssc.sample_size(
    treatment_weights=treatment_weights,
    mde_column="mde",
    nim_column="nim",
    preferred_direction_column="preference",

)

ss[['metric_name', 'adjusted_alpha_power_sample_size', 'null_hypothesis', 
    'alternative_hypothesis','required_sample_size_for_metric']]

,metric_name,adjusted_alpha_power_sample_size,null_hypothesis,alternative_hypothesis,required_sample_size_for_metric
0,unique_podcast_consume,0.0025,0.000000,0.1751,894112.0
1,wau_14d,0.0025,-0.001126,0.0000,2437730.0
2,podcast_consumed_14d,0.0025,-4.488800,0.0000,2674614.0
3,overall_consumed_14d,0.0025,-8.325000,0.0000,1687538.0


so the maximum sample size required for the experiment is 2437733 which is well within the avialable pool of 2742786 customers